In [1]:
#using Pkg; Pkg.add("ParallelDataTransfer")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed ParallelDataTransfer ─ v0.5.0
  Updating `~/.julia/environments/v1.2/Project.toml`
  [2dcacdae] + ParallelDataTransfer v0.5.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [2dcacdae] + ParallelDataTransfer v0.5.0


In [2]:
using Distributed
Distributed.addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
nworkers()

4

In [4]:
using Distributed
@everywhere using ParallelDataTransfer


┌ Info: Precompiling ParallelDataTransfer [2dcacdae-9679-587a-88bb-8b444fb7085b]
└ @ Base loading.jl:1242


In [5]:
@everywhere function rule30(ca::Array{Bool})
    lastv = ca[1]
    for i in 2:(length(ca)-1)
        current = ca[i]
        ca[i] = xor(lastv, ca[i] || ca[i+1])
        lastv = current
    end
end

In [6]:
@everywhere function getsetborder(ca::Array{Bool},
                                  neighbours::Tuple{Int64,Int64})
   ca[1] = (@fetchfrom neighbours[1] caa[end-1])
   ca[end] = (@fetchfrom neighbours[2] caa[2])
end


In [7]:
function printsimdist(workers::Array{Int})
    for w in workers
        dat = @fetchfrom w caa
        for b in dat[2:end-1]
            print(b ? "#" : " ")
        end
    end
    println()
end

printsimdist (generic function with 1 method)

In [8]:
function runca(steps::Int, visualize::Bool)
    @sync for w in workers()
        @async @fetchfrom w fill!(caa, false)
    end
    @fetchfrom wks[Int(nwks/2)+1] caa[2]=true
    visualize && printsimdist(workers())
    for i in 1:steps
        @sync for w in workers()
            @async @fetchfrom w getsetborder(caa, neighbours)
        end
        @sync for w in workers()
            @async @fetchfrom w rule30(caa)
        end
        visualize && printsimdist(workers())
    end
end

runca (generic function with 1 method)

In [9]:
wks = workers()
nwks = length(wks)
for i in 1:nwks
    sendto(wks[i],neighbours=(i==1 ? wks[nwks] : wks[i-1],
                              i==nwks ? wks[1] : wks[i+1]))
    fetch(@defineat wks[i] const caa = zeros(Bool,15+2));
end

In [10]:
runca(20,true)

                              #                             
                             ###                            
                            ##  #                           
                           ## ####                          
                          ##  #   #                         
                         ## #### ###                        
                        ##  #    #  #                       
                       ## ####  ######                      
                      ##  #   ###     #                     
                     ## #### ##  #   ###                    
                    ##  #    # #### ##  #                   
                   ## ####  ## #    # ####                  
                  ##  #   ###  ##  ## #   #                 
                 ## #### ##  ### ###  ## ###                
                ##  #    # ###   #  ###  #  #               
               ## ####  ## #  # #####  #######              
              ##  #   ##